## dataframe으로 저장된 opera trial balance를 이용해서, 메입카드사 별 금액과 건 수 정렬

In [201]:
import pandas as pnds
import pickle
import datetime

# dataframe으로 저장된 opera trial balance 읽기
yesterday = datetime.datetime.now() - datetime.timedelta(1) # 어제 날짜 추출
target_date = yesterday.strftime("%Y%m%d")                  # 어제 날짜 포맷
target_date = '20220710'

dfdata_dir = './dfdata/' + target_date + '/'
df_filename = 'df_kicc_history_' + target_date               # 파일 이름: df_kicc_history_YYYYMMDD
# 임시
df_filename = 'df_kicc_history_20220710'

try:
    with open(dfdata_dir + df_filename, "rb") as file:
        card_history_df = pickle.load(file)
except Exception as e:
    with open('./error.log', 'a') as file:
        file.write(
            f'[{__name__}.py] <{datetime.datetime.now()}> pickle file-reading error {dfdata_dir + df_filename} : {e}\n'
        )
        print(
            f'[{__name__}.py] <{datetime.datetime.now()}> pickle file-reading error {dfdata_dir + df_filename} : {e}'
        )
    raise(e)

card_history_df

,거래고유번호,승인구분,date,승인번호,카드번호,발급카드사,매입카드사,금액
546,260022371879,승인,2022-06-26,26129215,5365-1013-6565-3389,카카오뱅크체크,국민카드,33000
545,260133029316,승인,2022-06-26,31298110,4265-8692-1246-4648,KB 기업카드,국민카드,69000
544,260133030440,승인,2022-06-26,31298165,4265-8692-1246-4648,KB 기업카드,국민카드,69000
543,260221478806,승인,2022-06-26,31298200,4265-8692-1246-4648,KB 기업카드,국민카드,69000
542,260222480155,승인,2022-06-26,31298222,4265-8692-1246-4648,KB 기업카드,국민카드,69000
...,...,...,...,...,...,...,...,...
4,112312565531,승인,2022-07-11,899600,5557-9775-8327-1353,해외마스타,하나구외환,93822
3,120133603298,승인,2022-07-12,237300,5557-9774-3783-5437,해외마스타,하나구외환,107454
2,120133604990,승인,2022-07-12,092308,5567-1745-1565-3027,해외마스타,하나구외환,80388
1,120321819439,승인,2022-07-12,152896,5137-9680-0253-7341,해외마스타,하나구외환,125290


In [202]:
grouped_df = card_history_df.groupby(['매입카드사']).aggregate({'승인번호': 'count', '금액': 'sum'})
print(grouped_df)


       승인번호        금액
매입카드사                
NH카드     21   2878266
국민카드     75   8348054
롯데카드     13   1178689
비씨카드     82  15299957
삼성카드     62  10524706
신한카드     43   4688095
하나구외환   201  29084067
현대카드     32   4732623


In [203]:
# '발급카드사'를 바탕으로 EDI파일의 내용대로 분리
# '발급카드사' series를 looping => 조건식으로 분류리스트를 만들어서 컬럼으로 추가

# 카드분류 컬럼 추가
card_history_df['카드분류'] = '분류'        # 'LT', 'KEB', 'JCB', 'VISA', 'MASTER', 'SA', 'SS', 'SH', 'BC', 'KB', 'HD', 'NH', 'CITI'

# 'LT'
condition = (card_history_df['매입카드사'].str.startswith('롯데', na=False))
bc_cards = card_history_df[condition].index.tolist()
card_history_df.loc[bc_cards, '카드분류'] = '9244'

# 'KEB'
condition = (card_history_df['매입카드사'] == '하나구외환')  & (card_history_df['발급카드사'].str.startswith('하나', na=False) | card_history_df['발급카드사'].str.startswith('토스', na=False))
bc_cards = card_history_df[condition].index.tolist()
card_history_df.loc[bc_cards, '카드분류'] = '9243'

# 'JCB'
condition = (card_history_df['발급카드사'].str.contains('제이씨비', na=False))
bc_cards = card_history_df[condition].index.tolist()
card_history_df.loc[bc_cards, '카드분류'] = '9235'

# 'VISA'
condition = (card_history_df['발급카드사'].str.contains('해외비자', na=False))
bc_cards = card_history_df[condition].index.tolist()
card_history_df.loc[bc_cards, '카드분류'] = '9231'

# 'MASTER'
condition = (card_history_df['발급카드사'].str.contains('해외마스타', na=False))
bc_cards = card_history_df[condition].index.tolist()
card_history_df.loc[bc_cards, '카드분류'] = '9233'

# 'SA'
condition = (card_history_df['발급카드사'].str.contains('해외아멕스', na=False))
bc_cards = card_history_df[condition].index.tolist()
card_history_df.loc[bc_cards, '카드분류'] = '9232'

# 'SS'
condition = (card_history_df['발급카드사'].str.startswith('삼성', na=False)) & (card_history_df['발급카드사'].str.contains('비씨', na=False) == False)
bc_cards = card_history_df[condition].index.tolist()
card_history_df.loc[bc_cards, '카드분류'] = '9239'

# 'SH'
condition = (card_history_df['매입카드사'].str.startswith('신한', na=False))
bc_cards = card_history_df[condition].index.tolist()
card_history_df.loc[bc_cards, '카드분류'] = '9242'

# 'BC'
condition = (card_history_df['매입카드사'].str.startswith('비씨', na=False))      # 'CITI'씨티카드와 중복되지만, 'CITI'는 마지막에 다시 셋팅
bc_cards = card_history_df[condition].index.tolist()
card_history_df.loc[bc_cards, '카드분류'] = '9237'

# 'KB'
condition = (card_history_df['매입카드사'].str.startswith('국민', na=False))
bc_cards = card_history_df[condition].index.tolist()
card_history_df.loc[bc_cards, '카드분류'] = '9238'

# 'HD'
condition = (card_history_df['매입카드사'].str.startswith('현대', na=False))
bc_cards = card_history_df[condition].index.tolist()
card_history_df.loc[bc_cards, '카드분류'] = '9245'

# 'NH'
condition = (card_history_df['매입카드사'].str.startswith('NH', na=False))
bc_cards = card_history_df[condition].index.tolist()
card_history_df.loc[bc_cards, '카드분류'] = '9240'

# 'CITI'
condition = (card_history_df['발급카드사'].str.startswith('씨티', na=False)) & (card_history_df['발급카드사'].str.contains('비씨', na=False) == False)
bc_cards = card_history_df[condition].index.tolist()
card_history_df.loc[bc_cards, '카드분류'] = '9241'


# 어제와 오늘 날짜의 파일이 섞여 있는 현 원본을 따로 저장

In [204]:
# 그룹핑
grouped_df = card_history_df.groupby(['카드분류']).aggregate({'승인번호': 'count', '금액': 'sum'})
print(grouped_df)


      승인번호        금액
카드분류                
9231    17   4225538
9232     7   3969748
9233   160  21881357
9235     1    327250
9237    79  14822557
9238    75   8348054
9239    55   6554958
9240    21   2878266
9241     3    477400
9242    43   4688095
9243    23   2649922
9244    13   1178689
9245    32   4732623


In [205]:
map_code_edi = {
    # TRX_CODE : 엑셀좌표
    '9231': 'C7',
    '9232': 'C9',
    '9233': 'C8',
    '9234': 'C15',
    '9235': 'C6',
    '9236': 'C3',
    '9237': 'C12',
    '9238': 'C14',
    '9239': 'C10',
    '9240': 'C16',
    '9241': 'C17',
    '9242': 'C11',
    '9243': 'C5',
    '9244': 'C4',
    '9245': 'C15'
}
    
# 카드 트랙잭션 코드별 금액 리스트 딕셔너리
actual_cards_dict = {
    # {'transaction code': [금액,],}
    '9231': ['='],
    '9232': ['='],
    '9233': ['='],
    '9234': ['='],
    '9235': ['='],
    '9237': ['='],
    '9238': ['='],
    '9239': ['='],
    '9240': ['='],
    '9241': ['='],
    '9242': ['='],
    '9243': ['='],
    '9244': ['='],
    '9245': ['=']
}

# 코드별로 해당되는 각 리스트에 금액을 추가, 나중에 코드별로 join해서 edi 엑셀 파일에 저장(map_code_edi 참조)
for index, row in card_history_df.iterrows():                   # dataframe row looping
    actual_cards_dict[row['카드분류']].append(f"+{row['금액']}")     # 금액을 str 타입으로 바꾸어서 해당 리스트에 저장


In [206]:
import openpyxl
import os

# EDI 템플릿 엑셀 파일을 읽어들임
excel_filename = './EDI-xx월.xlsx'                                      # EDI 엑셀 템플릿 파일

try:
    edi_excel = openpyxl.load_workbook(excel_filename, data_only=False) # 수식파일 포함하여 엑셀파일 읽어 들임
    ws = edi_excel.active                                               # 활성화된 sheet
except Exception as e:
    with open('./error.log', 'a') as file:
        file.write(
            f'[{__name__}.py] <{datetime.datetime.now()}> openpyxl file-reading error {excel_filename} : {e}\n'
        )
        print(
            f'[{__name__}.py] <{datetime.datetime.now()}> openpyxl file-reading error {excel_filename} : {e}\n'
        )
    raise(e)

# 4. trial balance 내용을 순서대로 돌면서 엑셀에 입력
for code, amounts in actual_cards_dict.items():     # trx-code를 이용해서 ws와 opera_df에 접근하는 looping
    ws[map_code_edi[code]] = ''.join(amounts)
edi_excel.save('./arrange.xlsx')